In [99]:
import os
from openai import OpenAI

In [100]:
import requests
import json
from pydantic import BaseModel, Field

In [101]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [102]:
"""
docs: https://platform.openai.com/docs/guides/function-calling
"""

'\ndocs: https://platform.openai.com/docs/guides/function-calling\n'

#First we define the function or tool that we want to call

In [103]:
def get_weather(latitude,longitude):
    """Publicly available API that gives weather info from latitude and longitude."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

#Calling the model with get_weather tool defined

In [104]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },

    }
]
system_prompt = "You're a helpful weather assistant"

In [105]:
messages = [
    {"role": "system", "content": system_prompt },
    {"role": "user", "content": "What's the weather in Texas today?" },
]

In [106]:
completion = client.chat.completions.create(
    model= "gpt-4o-mini",
    messages= messages,
    tools= tools,
)

#Model decides to call the function

In [107]:
completion.model_dump()

{'id': 'chatcmpl-CEiHIuZddVUMMQKRU0FZ0j15v6EHd',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_SY4ZmvW0GdX8KssBcXBiaKR0',
      'function': {'arguments': '{"latitude": 31.9686, "longitude": -99.9018}',
       'name': 'get_weather'},
      'type': 'function'}]}}],
 'created': 1757622832,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_8bda4d3a2c',
 'usage': {'completion_tokens': 41,
  'prompt_tokens': 64,
  'total_tokens': 105,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

#Execute get_weather function

In [108]:
assistant_messages = completion.choices[0].message
messages.append(assistant_messages)

In [109]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)

In [110]:
for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    # messages.append(completion.choices[0].message)

    result = call_function(name,args)

    messages.append(
        {"role":"tool", 
         "tool_call_id": tool_call.id, 
         "content": json.dumps(result),
         }
    )

#Transfer result and call model again

In [111]:
class WeatherResponse(BaseModel):
    temperature: float = Field(
        description= "Current temperature in Celsius for given location."
    )
    response: str = Field(
        description= "A natural language response to user's question."
    )

In [112]:
completion2 = client.beta.chat.completions.parse(
    model = "gpt-4o-mini",
    messages = messages,
    # tools = tools,
    response_format = WeatherResponse,
)

In [113]:
final_response = completion2.choices[0].message.parsed

In [114]:
final_response.temperature

33.7

In [115]:
final_response.response

'The current temperature in Texas is 33.7°C with a wind speed of 15.1 km/h.'